In [4]:
from scipy.io import wavfile
import numpy as np
import os

import random
from random import randint
import math



def rip_audio(filename):
  f = len(filename[1])
  p = np.zeros(f)
  # Mix the audio down to mono ha
  try:
    len(filename[1][0])
    for i in range(f):
      p[i] = (filename[1][i][1] + filename[1][i][0])/2
  except Exception:
    for i in range(f):
      p[i] = filename[1][i]

  return p

def scrambler(audio_array, number_of_swaps):
    f = len(audio_array)

    sample_length = 88200
    temp_array = [0] * sample_length
    sample_start = randint(0,f - sample_length)

    # This loads in a sample of set length into a temp array
    for i in range(0, sample_length):
      temp_array[i] = audio_array[sample_start + i]

    for i in range(number_of_swaps):
      sample_length = randint(3000,88200)
      sample_start = randint(0,f)
      #print("start at: %s, length: %s\n" % (sample_start, sample_length))
      j = 0
      while(((sample_start + j)<f) and (j < sample_length)):
        temp = audio_array[sample_start+j]
        audio_array[sample_start+j] = temp_array[j]
        temp_array[j] = temp
        j = j + 1
    return audio_array

# audio_file is an np array do not be decieved
def delay(audio_file, time_seconds, decay_factor):
    track_length = len(audio_file)
    delay_tape = np.zeros(math.floor(44100 * time_seconds))
    decay_length = len(delay_tape)
    for i in range(track_length):
      audio_file[i] = (audio_file[i] + delay_tape[i % decay_length]) / (1 + decay_factor)
      delay_tape[i % decay_length] = audio_file[i] * decay_factor
    
    return audio_file

def fake_vibrato(audio_file, depth, rate):
  i = 0
  while (i < len(audio_file)):
        audio_file[i] = audio_file[i] * ((1 - (depth/2)) + (np.sin(np.pi * 2 * (rate * i/44100)) * depth))
        i = i + 1
  return audio_file

# factor is factor^-1 btw ;)
def bitcrush(audio_file, factor):
    if (factor < 1):
        return audio_file
    
    sample = audio_file[0]
    count = 0
    for i in range(len(audio_file)):
        if (count == factor):
            sample = audio_file[i]
            count = 0
            continue
        audio_file[i] = sample
        count += 1
    return audio_file

def chipmunkify(audio_file, multiple):
    if(multiple < 0):
        print("No negative numbers")
        return audio_file
    
    output_file = np.zeros(math.ceil(len(audio_file) * multiple))
    count = 0
    for i in range(len(output_file)):
        if (count >= len(audio_file)):
            break
        output_file[i] = audio_file[math.floor(count)]
        count = count + multiple
    return output_file

def scramblebitcrush(audio_file, factor, number_of_swaps):
    f = len(audio_array)

    sample_length = 88200
    temp_array = [0] * sample_length
    sample_start = randint(0,f - sample_length)

    # This loads in a sample of set length into a temp array
    for i in range(0, sample_length):
      temp_array[i] = audio_array[sample_start + i]

    for i in range(number_of_swaps):
      sample_length = randint(3000,88200)
      sample_start = randint(0,f)
      if (randint(0,2) == 1):
        temp_array = bitcrush(temp_array,factor)
      #print("start at: %s, length: %s\n" % (sample_start, sample_length))
      j = 0
      while(((sample_start + j)<f) and (j < sample_length)):
        temp = audio_array[sample_start+j]
        audio_array[sample_start+j] = temp_array[j]
        temp_array[j] = temp
        j = j + 1
    return audio_array



In [5]:
#Upload file here
# This HAS to be Mono if you want to mix stereo down here's how

audiofile = files.upload()
keys = list(audiofile.keys())
inputfile = wavfile.read(keys[0])
print(inputfile[0])

NameError: name 'files' is not defined

In [6]:
inputfile = wavfile.read("resources/input.wav")

<ipython-input-6-7104168dfb73>:1: WavFileWarning: Chunk (non-data) not understood, skipping it.
  inputfile = wavfile.read("resources/input.wav")


In [7]:
# reset
audio_array = rip_audio(inputfile)
print(audio_array)

<ipython-input-4-91c5fd971ab3>:18: RuntimeWarning: overflow encountered in short_scalars
  p[i] = (filename[1][i][1] + filename[1][i][0])/2


[  0.    0.    0.  ... 550.  543.5 546. ]


In [61]:
#Scrambler
audio_array = scrambler(audio_array,400)

In [31]:
# "Vibrato"
audio_array = fake_vibrato(audio_array,0.69,400)

In [ ]:
#Delay
audio_array = delay(audio_array, 1, 0.6)

In [49]:
#Bicrush
audio_array = bitcrush(audio_array,3)

In [50]:
#Chipmunkify
audio_array = chipmunkify(audio_array,0.9)

In [57]:
#ScrambleBitcrush
audio_array = scramblebitcrush(audio_array, 3, 400)

In [62]:
# Get the file
wavfile.write("resources/new_{}".format("newfile.wav"),inputfile[0],audio_array.astype(np.int16))